In [0]:
!pip install -q kaggle
!pip install -q simpletransformers
!mkdir .kaggle

import json
token = {"username":"abdualimov","key":"f6a217b16ab63fb20f2ae6db2fe4e418"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
print('Скачиваю датасеты')
!kaggle competitions download -c sf-dl-movie-genre-classification -p /content
!ls

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
Скачиваю датасеты
 65% 9.00M/13.8M [00:00<00:00, 15.1MB/s]
100% 13.8M/13.8M [00:00<00:00, 23.5MB/s]
 37% 5.00M/13.7M [00:00<00:01, 8.57MB/s]
100% 13.7M/13.7M [00:00<00:00, 21.6MB/s]
 65% 9.00M/13.8M [00:00<00:00, 17.0MB/s]
100% 13.8M/13.8M [00:00<00:00, 25.6MB/s]
 36% 5.00M/13.9M [00:00<00:00, 9.79MB/s]
100% 13.9M/13.9M [00:00<00:00, 23.7MB/s]
  0% 0.00/718k [00:00<?, ?B/s]
100% 718k/718k [00:00<00:00, 47.5MB/s]
sample_data	       test.csv.zip	  train.csv.zip
sample-submission.csv  test_data.txt.zip  train_data.txt.zip


In [0]:
import random
import numpy as np 
import pandas as pd 

from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score

In [0]:
train = pd.read_csv('train.csv.zip')
test = pd.read_csv('test.csv.zip')
submission = pd.read_csv('sample-submission.csv')

train['new_str'] = train['name'] + train['text']
train = train[['new_str', 'genre']]

test['text'] = test['name'] + test['text']
test.drop(['name'], axis = 1, inplace = True)

classes =  list(set(train.genre))
b = {}
a = 0
for i in classes:
  b[i] = a
  a += 1
train['genre'] = train['genre'].map(b)

In [0]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=27, args={"num_train_epochs": 2,
                                                                              "fp16": False, 
                                                                              'overwrite_output_dir': True})

In [0]:
model.train_model(train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:251: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 3.286793

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Running loss: 3.122003

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.535772

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.932510


Running loss: 0.598942



In [0]:
pred, raw_outputs = model.predict(test['text'])
p = {v:k for k, v in b.items()}
submission['genre'] = pred
submission['genre'] = submission['genre'].map(p)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
submission.to_csv('/content/drive/My Drive/text_class_submission_10.csv', index=False)

In [0]:
len(raw_outputs[0])

27

In [0]:
pred[0]

2

In [0]:
raw_outputs[0]

array([-1.6008345 , -2.4814446 ,  5.2900405 , -3.8651638 , -0.55568933,
       -3.0567167 , -2.8520112 , -0.06716382,  0.65079314,  0.41097426,
        1.9081004 , -3.4103076 , -1.7526495 , -1.6024286 , -1.8973329 ,
        0.35614672,  2.1244707 ,  1.6331499 , -0.44760334, -1.6375322 ,
       -2.3757231 , -0.8964006 , -0.36394703, -2.785734  ,  3.4852092 ,
        4.8531275 ,  0.5530864 ], dtype=float32)

In [0]:
predict_proba = pd.DataFrame(raw_outputs, columns=classes)

In [0]:
predict_proba.to_csv('/content/drive/My Drive/pred_proba_roberta.csv', index=False)